York Solutions | AML BB Cohort 2023-2024
Final Capstone Project - Customer Propensity

This assignment is to design, build, evaluate, and deploy a Machine Learning model to predict customer propensity to perform the "Add to Cart" action. The model should accurately predict whether this action will be performed or not based on the features selected or developed from the given dataset. This model is essential as part of the Q2 business goal to increase the return on ad-spending of the company employing google analytics. 

Using google analytics 360 sample dataset on BigQuery, develop, and deploy a model in VertexAI to predict when a 'hit' will have an event action of "Add to Cart". To start with, a 1-month period of my choice should be used to train the model, with the goal of being able to easily substitute in the 12-month period (full dataset) after validating the model. Because of the dataset size, it is assumed that data cleaning and feature selection will take place to test and train the modle in a timely manner. 



scikit package, seaborn, numpy? Are there any terminal installation and commands-required?


In [1]:
## Allowed dependencies for this project that could still be included:
## SciPy
## Google-cloud-*

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel



Data Exploration and Pre-Processing:

1. Load the dataset from BigQuery into this Jupyter Notebook | Vertex AI | Python Environment (1-month worth of data)  -- DONE
2. Handle Missing Values, outliers, and any data quality issues  
3. Perform Exploratory Data Analysis to understand the structure, distribution, and characteristics of the data
4. Convert categorical variables into numerical representations if needed (e.g., one-hot encoding)
5. Split the data into training and test sets


In [2]:
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

BQ_LOCATION = 'US'
REGION = 'us-east-1' ## Have to double check location

In [3]:
%%bigquery selectData_bq_df --project $PROJECT_ID
SELECT 
concat(fullVisitorId, visitId) AS session_id,
visitNumber, CAST(SUBSTRING(`date`,5,2) AS int) AS month,
CAST(SUBSTRING(`date`,7,2) AS int) AS day_of_month,
totals.hits,
hit.hitNumber,
CASE WHEN totals.visits IS NULL THEN 0 ELSE 1 END AS visits,
CASE WHEN totals.bounces IS NULL THEN 0 ELSE 1 END AS bounces, 
CASE WHEN trafficSource.isTrueDirect IS NULL THEN 0 WHEN trafficSource.isTrueDirect IS FALSE THEN 0 ELSE 1 END AS isTrueDirect,
trafficSource.medium,
trafficSource.source,
CASE socialEngagementType WHEN 'Not Socially Engaged' THEN 0 ELSE 1 END AS social_Engagement,
channelGrouping,
device.deviceCategory,
promo.promoName,
CASE WHEN hit.promotionActionInfo.promoIsView IS NULL THEN 0 ELSE 1 END AS promoIsView,
CASE hit.eCommerceAction.action_type WHEN '3' THEN 1 ELSE 0 END AS add_to_cart
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  LEFT JOIN UNNEST (hits) AS hit
  LEFT JOIN UNNEST (hit.promotion) AS promo
WHERE 
  _TABLE_SUFFIX BETWEEN '20161120' AND '20161220'
ORDER BY session_id, hit.hitNumber


Query is running:   0%|          |

Downloading:   0%|          |

In [10]:
#Pivoting the PromoName to create new promoColumns, filled with 1s and 0s

# Mapping from promoName to new column names

promo_mapping = {
    "Office": "promoOffice",
    "Leatherette Journal": "promoLeather",
    "Electronics Accessory Pouch": "promoElectro",
    "Mens T-Shirts": "promoMtshirts",
    "Fruit Games": "promoFruit",
    "Andriod Brand": "promoAndroid",
    "Drinkware": "promoDrinks",
    "Office Campaign": "promoOfficeCampaign",
    "Pen Pencil & Highlighter Set": "promoPenPencilHigh",
    "Lifestyle Campaign": "promoLifestyle",
    "Accessories Campaign": "promoAccessories",
    "Engraved Ceramic Google Mug": "promoEngravedCeramic",
    "Backpacks": "promoBack",
    "Womens T-Shirts": "promoWtshirts",
    "Google Brand": "promoGoogle",
    "YouTube Brand": "promoYoutube",
    "Apparel": "promoApparel",
    "Apparel Campaign": "promoApparelCampaign"
}


# Loop through the promo_mapping and create new columns, taking into account None values
for promo_name, new_column_name in promo_mapping.items():
    selectData_bq_df[new_column_name] = selectData_bq_df['promoName'].apply(lambda x: 1 if x is not None and promo_name in x else 0)

    
# Drop the original promoName column
selectData_bq_df.drop(columns=['promoName'], inplace=True)


# If you want to fill NaN values with 0 in the new columns
selectData_bq_df.fillna(0, inplace=True)




In [11]:
## Display the first 10 rows of the Dataframe to inspect the data, confirm appropriate structure to dataset transfer:

print(selectData_bq_df.head(10))


                      session_id  visitNumber  month  day_of_month  hits  \
0  00000204243422487471480578901            1     11            30    17   
1  00000204243422487471480578901            1     11            30    17   
2  00000204243422487471480578901            1     11            30    17   
3  00000204243422487471480578901            1     11            30    17   
4  00000204243422487471480578901            1     11            30    17   
5  00000204243422487471480578901            1     11            30    17   
6  00000204243422487471480578901            1     11            30    17   
7  00000204243422487471480578901            1     11            30    17   
8  00000204243422487471480578901            1     11            30    17   
9  00000204243422487471480578901            1     11            30    17   

   hitNumber  visits  bounces  isTrueDirect  medium  ... promoPenPencilHigh  \
0          1       1        0             0  (none)  ...                  0   
1    

In [12]:
## DataFrame Column Data Types

selectData_bq_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128153 entries, 0 to 1128152
Data columns (total 34 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   session_id            1128153 non-null  object
 1   visitNumber           1128153 non-null  Int64 
 2   month                 1128153 non-null  Int64 
 3   day_of_month          1128153 non-null  Int64 
 4   hits                  1128153 non-null  Int64 
 5   hitNumber             1128153 non-null  Int64 
 6   visits                1128153 non-null  Int64 
 7   bounces               1128153 non-null  Int64 
 8   isTrueDirect          1128153 non-null  Int64 
 9   medium                1128153 non-null  object
 10  source                1128153 non-null  object
 11  social_Engagement     1128153 non-null  Int64 
 12  channelGrouping       1128153 non-null  object
 13  deviceCategory        1128153 non-null  object
 14  promoIsView           1128153 non-null  Int64 
 15

In [13]:
## Print out the first and last five rows to better visualize the data

print(selectData_bq_df)

                            session_id  visitNumber  month  day_of_month  \
0        00000204243422487471480578901            1     11            30   
1        00000204243422487471480578901            1     11            30   
2        00000204243422487471480578901            1     11            30   
3        00000204243422487471480578901            1     11            30   
4        00000204243422487471480578901            1     11            30   
...                                ...          ...    ...           ...   
1128148  99998874200163075701480139386            1     11            25   
1128149  99998874200163075701480139386            1     11            25   
1128150  99998874200163075701480139386            1     11            25   
1128151  99998874200163075701480139386            1     11            25   
1128152  99998874200163075701480139386            1     11            25   

         hits  hitNumber  visits  bounces  isTrueDirect  medium  ...  \
0          17  

In [14]:
## Summary Statistics for numerical fields in out DataFrame

selectData_bq_df.describe()

,visitNumber,month,day_of_month,hits,hitNumber,visits,bounces,isTrueDirect,social_Engagement,promoIsView,...,promoPenPencilHigh,promoLifestyle,promoAccessories,promoEngravedCeramic,promoBack,promoWtshirts,promoGoogle,promoYoutube,promoApparel,promoApparelCampaign
count,1128153.0,1128153.0,1128153.0,1128153.0,1128153.0,1128153.0,1128153.0,1128153.0,1128153.0,1128153.0,...,1128153.0,1128153.0,1128153.0,1128153.0,1.128153e+06,1.128153e+06,1.128153e+06,1.128153e+06,1.128153e+06,1128153.0
mean,2.60915,11.621219,15.778646,15.724808,7.465763,1.0,0.089061,0.340683,0.0,0.652316,...,0.0,0.0,0.0,0.0,7.355119e-02,7.302733e-02,7.404935e-02,7.400592e-02,7.397312e-02,0.0
std,10.243455,0.485084,8.93807,24.377132,14.397759,0.0,0.284831,0.473939,0.0,0.476235,...,0.0,0.0,0.0,0.0,2.610392e-01,2.601815e-01,2.618513e-01,2.617806e-01,2.617273e-01,0.0
min,1.0,11.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,1.0,11.0,8.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,1.0,12.0,16.0,7.0,2.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
75%,2.0,12.0,23.0,18.0,7.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
max,286.0,12.0,30.0,286.0,286.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0


In [15]:
## Check for Missing Values

selectData_bq_df.isnull().sum()


session_id              0
visitNumber             0
month                   0
day_of_month            0
hits                    0
hitNumber               0
visits                  0
bounces                 0
isTrueDirect            0
medium                  0
source                  0
social_Engagement       0
channelGrouping         0
deviceCategory          0
promoIsView             0
add_to_cart             0
promoOffice             0
promoLeather            0
promoElectro            0
promoMtshirts           0
promoFruit              0
promoAndroid            0
promoDrinks             0
promoOfficeCampaign     0
promoPenPencilHigh      0
promoLifestyle          0
promoAccessories        0
promoEngravedCeramic    0
promoBack               0
promoWtshirts           0
promoGoogle             0
promoYoutube            0
promoApparel            0
promoApparelCampaign    0
dtype: int64

In [ ]:
# Handle Missing Values: None Exist from Data Structuring and Modifications

## Fill missing values with the mode values: Not Applicable

In [ ]:
## Seaborn EDA 

sns.pairplot(selectData_bq_df)
plt.show()


/opt/conda/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/opt/conda/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/opt/conda/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/opt/conda/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  

Pipeline Pre-Processing & Feature Selection / Engineering:

1. Select relevant features that might influence the "Add to Cart" action
2. Perform Feature Engineering to create new features if necessary (e.g. combining or transforming existing features)
3. Use domain knowledge and insights gained from EDA to guide feature selection and engineering

In [ ]:
# One-Hot Encoding

# Min-Max Scaling

# Z-Score Scaling

# Or really, any other pre-processing that may be pragmatic.. 

In [ ]:
## Feature Selection -- DO NOT SELECT ALL, OTHERWISE IT'S OVERFITTING

selected_features = [
    
    # Original Data Features Chosen
    
    # Any Other Modified Features?


]

Model Selection and Training:

1. Choose appropriate machine learning models for binary classification (e.g. Logistic Regression, Random Forest, GB)
2. Train different models on the training dataset
3. Evaluate models using appropriate evaluation metrics (e.g. accuracy precision, recall, F1-score)
4. Optimize hyperparameters using techniques like grid search or random search


In [ ]:
## Split the Data into features (X) and Target Variable (y)
X = df[selected_features]
y = df[add_to_cart]     # the target is the eCommerce action_type "Add to Cart"

In [ ]:
## Split the data into training and testing sets (75% training, 25% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=72)

Model Evaluation and Tuning: 

1. Evaluate the trained models on the testing dataset
2. Analyze model performance using evaluation metrics and confusion matrices
3. Fine-tune the models based on the evaluation results
4. Iterate on feature selection, engineering, and model training if necessary


In [ ]:
## Placeholder

Deployment:

1. Deploy the trained model to Google Cloud Platform (GCP) using VertexAI or any other suitable service
2. Document the deployment process, including all necessary steps for redeployment.
3. Ensure the deployed model can respond to prediction requests efficiently

In [ ]:
## Placeholder

Project Management and Documentation:

1. Manage code using Git and maintain version control.
2. Create a well-organized repository following the naming convention specied in the requirements
3. Document all code, decisions, and methodologies used throughout the project, the more the better...
4. Generate and analysis report covering EDA, data pre-processing, feature selection, model choice, evaluation metrics, model tuning, and performance monitoring

In [ ]:
## Placeholder

Cost Management: 

1. Keep track of the resources used during the project, inlcuding compute resources fro training and deploying the model
2. Analyse the project cost and identify opportunities for optimization
3. Docuemtn the total cost to deploy the model, including any additional cost monitoring or resource optimization done